# AssociationTable with word embeddings target - association

- word2vec or Glove?
-> Glove because whole corpus relations not only neighboring, useful when dealing with associations? 

In [1]:
# getting information on the usde model in gensim
import gensim.downloader as api
import json
info = api.info()
print(json.dumps(info, indent=4))

for corpus_name, corpus_data in sorted(info['corpora'].items()):
    print(
        '%s (%d records): %s' % (
            corpus_name,
            corpus_data.get('num_records', -1),
            corpus_data['description'][:40] + '...',
        )
    )

{
    "corpora": {
        "semeval-2016-2017-task3-subtaskBC": {
            "num_records": -1,
            "record_format": "dict",
            "file_size": 6344358,
            "reader_code": "https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py",
            "license": "All files released for the task are free for general research use",
            "fields": {
                "2016-train": [
                    "..."
                ],
                "2016-dev": [
                    "..."
                ],
                "2017-test": [
                    "..."
                ],
                "2016-test": [
                    "..."
                ]
            },
            "description": "SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collect

"glove-wiki-gigaword-100": {
            "num_records": 400000,
            "file_size": 134300434,
            "base_dataset": "Wikipedia 2014 + Gigaword 5 (6B tokens, uncased)",
            "reader_code": "https://github.com/RaRe-Technologies/gensim-data/releases/download/glove-wiki-gigaword-100/__init__.py",
            "license": "http://opendatacommons.org/licenses/pddl/",
            "parameters": {
                "dimension": 100
            },
            "description": "Pre-trained vectors based on Wikipedia 2014 + Gigaword 5.6B tokens, 400K vocab, uncased (https://nlp.stanford.edu/projects/glove/).",
            "preprocessing": "Converted to w2v format with `python -m gensim.scripts.glove2word2vec -i <fname> -o glove-wiki-gigaword-100.txt`.",
            "read_more": [
                "https://nlp.stanford.edu/projects/glove/",
                "https://nlp.stanford.edu/pubs/glove.pdf"
            ],
            "checksum": "40ec481866001177b8cd4cb0df92924f",
            "file_name": "glove-wiki-gigaword-100.gz",
            "parts": 1
        },

In [1]:
# creqating tarrget:assoc dataFrame (wit CS) with cosine similarities as values of targets and acossiations embedding vectors distances
import pandas as pd
import numpy as np
import gensim.downloader

glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100') # type KeyedVectors<vector_size=100, 400000 keys>

#datatype= 'extreme'
datatype= 'filtered'
#wordclass = 'adj'
wordclass = 'nouns'
#wordclass = 'verbs'


#creating datatable to fill with cosine distances - assoc/targets without embeddings are deleted from dataframe
df = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_'+datatype+'/R1only_types_'+wordclass+'_filtered_AssocTable.csv')

#df = df.loc[:20,:'resolve'] #try slice

targets = list(df['Unnamed: 0'])
assoc = list(df.columns)
cs = df['CS']
assoc.remove('Unnamed: 0')
assoc.remove('CS')

df = df.select_dtypes(include=['number']) != 0  #getting truth values for nonzero
df['target'] = targets  #adding back targets to nonzero truth table

#looping through assoc to find out which ones have embedding
removelist = []
for a in assoc:
    try:
        glove_vectors[a]
    except KeyError:   #if assoc has no embedding
        removelist.append(a)
        continue
assoc = list(set(assoc) - set(removelist)) #remove assoc withut embeddings from assoc list
print('Dropped Associations: ' +str(len(removelist)))

embTable = pd.DataFrame(np.zeros((len(list(df.index)), len( ['Unnamed: 0','CS']+assoc))),index= list(df.index), columns = ['Unnamed: 0','CS']+assoc)
embTable['CS']=cs
embTable['Unnamed: 0'] = targets

#looping through targets and their associations for getting embeddings and calculating cosine distance when in original AssocTable non-zero value 
droppedtargets = 0

for t in targets:
   # print('Target: '+str(t))
    try: #trying to find embedding for target
        glove_vectors[t]  # numpy vector of target
        ind = embTable[embTable['Unnamed: 0'] == t].index[0]
        for a in assoc:
           # print('Assoc: '+str(a))
            nonzero = df.loc[ind,a]  #checking if t is associated with a
            if nonzero == True:
                embTable.loc[ind,a] = round(glove_vectors.distance(t,a),3) #adding cosine distance to embTable, rounded
    except KeyError:  #if target has no embedding
        embTable = embTable.drop(embTable.loc[embTable['Unnamed: 0'] == t]) #delete target from dataframe
        droppedtargets += 1
        print('Dropped Target: ' + str(t))
        continue
                                  
print('Amount dropped Targets:' + str(droppedtargets)) 
    
embTable.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/M3_CosDisTable/R1only_types_'+wordclass+'_'+datatype+'_CosDisTable.csv',index=False)
print(embTable)


Dropped Associations: 11257
Amount dropped Targets:0
      Unnamed: 0    CS  bonnet  tessellation  retiree  architecture  fun  \
0          music  4.31     0.0           0.0      0.0           0.0  0.0   
1            way  2.34     0.0           0.0      0.0           0.0  0.0   
2           none  2.59     0.0           0.0      0.0           0.0  0.0   
3          paper  4.93     0.0           0.0      0.0           0.0  0.0   
4           town  4.64     0.0           0.0      0.0           0.0  0.0   
...          ...   ...     ...           ...      ...           ...  ...   
3693     glamour  2.11     0.0           0.0      0.0           0.0  0.0   
3694  windscreen  4.59     0.0           0.0      0.0           0.0  0.0   
3695       liner  4.20     0.0           0.0      0.0           0.0  0.0   
3696     roadway  4.67     0.0           0.0      0.0           0.0  0.0   
3697       lipid  3.04     0.0           0.0      0.0           0.0  0.0   

      valuable  email  shall  ... 

In [ ]:
# wy cosine distance over1? because 1-similarity and similarity is cosine which goes from -1 to 1

# testing cosisen distance measure
import pandas as pd
import numpy as np
import gensim.downloader

#df = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/M3_CosDisTable/NORMnoNAtypes_adj_filtered_CosDisTable.csv')
sorted(((value, index) for index, value in enumerate(list(df.values[0][2::]))), reverse=True)
# (1.142, 8679),
 #(1.045, 11348),
 #(1.004, 15684)
# real - 'uninvolved' 1.14 index=8679
# real - 'wordplay'  1.004, index=15684)
list(df.columns)[15684]

w1 = 'know'
w2 = 'reveal'
w3 = 'real'
w4 = 'uninvolved'

glove_vectors = gensim.downloader.load('glove-wiki-gigaword-100')

print('Distance: '+w1+'-'+w2+': ' + str(glove_vectors.distance(w1,w2)))
print('Similarity: ' +w1+'-'+w2+': '+ str(glove_vectors.similarity(w1,w2)))

print('Distance: '+w3+'-'+w4+': ' + str(glove_vectors.distance(w3,w4)))
print('Similarity: ' +w3+'-'+w4+': '+ str(glove_vectors.similarity(w3,w4)))

#know - reveal : dist=0.446 sim = 0.55
#assimilate - mainstream : dist=0.727, sim = 0.273


# Rounding CosDis Values to save memory

In [ ]:
#rounding cosdis values to lower memory (if not fixed in associationtable code)
import pandas as pd
import numpy as np

#datatype= 'extreme'
datatype= 'filtered'
#wordclass = 'adj'
wordclass = 'nouns'
#wordclass = 'verbs'

rounding = 3

df = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/M3_CosDisTable/NORMnoNAtypes_'+wordclass+'_'+datatype+'_CosDisTable.csv')
#df = df.loc[:20,:'resolve'] #try slice
df = np.round(df, decimals = rounding)
df.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/M3_CosDisTable/NORMnoNAtypes_'+wordclass+'_'+datatype+'_CosDisTable.csv',index=False)
print(df)


In [ ]:
# dont know what I tested here

df.loc[df[df['target'] == 'know'].index[0],'reveal']

# association table with only R1 associations and T-A

In [ ]:
# gettig target-association table with 1 for associated R1 for target
import re
import pandas as pd

# TokenassociationTables
#  [2] new: SIMnoun_500 as pandas
#V2.2 itterating through associations and built table 

target_corpus = "/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/targets_Brysbaert_filtered_nouns.txt"
#target_corpus = "/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/SIMnoun_mini.txt"  #try-mini corpus
targetdata = pd.read_csv(target_corpus, sep='\t')

assoc_corpus = "/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/SWOW-EN.R100.csv"
data = {"ASSOC":['CS']}

#itterate through assocs
with open(assoc_corpus, "r") as asslist:
    for assline in asslist:
        assline = re.sub(r'"', "", assline)  #access SWOWtargets
        ass = assline.split(",")[9]       
        ass = re.sub(r'\n', "", ass) 
#itterate through targets and check if ass is in targets
        if ass in targetdata['TARGET'].values: 
            assind = targetdata[targetdata['TARGET']== ass].index.item() #accessing target index
            cs = targetdata['CS'].values[assind]  #accessing target CS
            data[ass] = [cs]                      #adding cs to target
            ass1 = assline.split(",")[10]  #access SWOWassocs + cleaning
            ass1 = re.sub(r'\n', "", ass1) 
            #ass2 = assline.split(",")[11]
            #ass2 = re.sub(r'\n', "", ass2) 
            #ass3 = assline.split(",")[12]
            #ass3 = re.sub(r'\n', "", ass3) 
            if ass1 not in data['ASSOC']: #assign SWOWassocs to colunns 
                data["ASSOC"].append(ass1)
           #if ass2 not in data['ASSOC']:
            #    data["ASSOC"].append(ass2)
            #if ass3 not in data['ASSOC']:
            #    data["ASSOC"].append(ass3)
df = pd.DataFrame.from_dict(data, orient ='index')
df = df.replace([None], 0)
print('DONE, Next Step :)')


In [ ]:
#  [3] Tokenassociation Tables
#filling existent pandas(all values zero) with counts
ASSOClist = list(data['ASSOC']) #get list of associations

with open(assoc_corpus, "r") as assoclist:
    for assocline in assoclist:
            assocline = re.sub(r'"', "", assocline)  #access assoctarget
            ass = assocline.split(",")[9]       
            ass = re.sub(r'\n', "", ass)
            if ass in data:
                ass1 = assocline.split(",")[10]  #access R1assocs + cleaning
                ass1 = re.sub(r'\n', "", ass1)
                assind = ASSOClist.index(ass1)          #getting column index
                df.loc[ass,assind] += 1                 #count target-assoc pair
                #ass2 = assocline.split(",")[11]  #access R2assocs + cleaning
                #ass2 = re.sub(r'\n', "", ass2)
                #assind = ASSOClist.index(ass2)          
                #df.loc[ass,assind] += 1
                #ass3 = assocline.split(",")[12]  #access R3assocs + cleaning
                #ass3 = re.sub(r'\n', "", ass3)
                #assind = ASSOClist.index(ass3)          
                #df.loc[ass,assind] += 1
#assocs as header 
df.columns = df.iloc[0]
df = df[1:]         
df.to_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/original_data/targets_Brysbaert_filtered/R1only_nouns_filtered_AssocTable.csv') 



In [9]:
df2 = pd.read_csv('/compLing/students/hiwi-theses/data/aylin.wahl/MUDCATData/M3_CosDisTable/R1only_types_adj_filtered_CosDisTable.csv')
df2


,Unnamed: 0,CS,reproductive,written,firefighter,favor,unit,remember,champaign,objector,...,farming,heathen,grave,gr,distracting,creation,glittering,timed,nonfat,treat
0,real,2.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,grand,1.67,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,recent,2.50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,simple,1.62,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,possible,1.56,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1364,marital,2.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1365,redundant,2.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1366,restricted,2.83,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1367,lyrical,2.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# prpcess the table from above
# lemmatize assocaitions: AssociationTable_M1_M2.ipynb -> R1only_types_adj_filtered_AssocTable.csv
# no NAs: AssociationTable_M1_M2.ipynb -> no NAs in R1s 
# NORM: not necessary for cosine distance 
# getting T-A embeddings for R1 only -> code above

#print(df.sum(axis = 1).nsmallest(n=10)) (testing the association tables)
